# Random Forest - Oversampling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import classification_report,confusion_matrix #import Confusion Matrix
from sklearn.model_selection import train_test_split # Splitting the data
from sklearn import preprocessing # Normalizing

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from IPython.display import Image  
from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz
import time


In [2]:
df_o = pd.read_csv("../data/processed/data_oversampled.csv",index_col=0)

In [3]:
# Normalizing the data
x = df_o.values #returns a numpy array
col = df_o.columns
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_n = pd.DataFrame(x_scaled, columns = col)

# df_n

,X,Y,S,A,Orientation,Dir,Season,Distance,HomeScoreBeforePlay,VisitorScoreBeforePlay,DefendersInTheBox,YardsToTouchdown,PlayerHeightInches,PlayerWeight,Age,Week,StadiumTypeCode,isFirstDown,Cornerbacks,DefensiveEnds
0,0.599181,0.503250,0.361275,0.163834,0.585278,0.867408,0.0,0.473684,0.000000,0.000000,0.444444,0.571429,0.444444,0.305556,0.266667,0.0000,0.000000,0.0,0.500000,0.6
1,0.395175,0.275784,0.681228,0.164514,0.391167,0.616671,0.0,0.473684,0.000000,0.000000,0.555556,0.346939,0.444444,0.305556,0.266667,0.0000,0.000000,0.0,0.500000,0.6
2,0.225216,0.502119,0.460449,0.171992,0.095194,0.438733,0.0,0.473684,0.122807,0.000000,0.555556,0.755102,0.555556,0.458333,0.066667,0.0000,0.000000,0.0,0.333333,0.2
3,0.273373,0.441650,0.566706,0.137322,0.415917,0.566296,0.0,0.473684,0.122807,0.000000,0.666667,0.183673,0.444444,0.444444,0.400000,0.0000,0.000000,0.0,0.333333,0.6
4,0.159217,0.454366,0.430933,0.080218,0.571111,0.783829,0.0,0.000000,0.122807,0.000000,0.777778,0.091837,0.555556,0.375000,0.333333,0.0000,0.000000,0.0,0.333333,0.6
5,0.084843,0.364510,0.563164,0.210741,0.044806,0.360961,0.0,0.473684,0.122807,0.000000,0.555556,0.908163,0.555556,0.458333,0.066667,0.0000,0.000000,0.0,0.333333,0.2
6,0.440510,0.522464,0.429752,0.178790,0.059806,0.392498,0.0,0.473684,0.122807,0.000000,0.555556,0.500000,0.555556,0.458333,0.066667,0.0000,0.000000,0.0,0.333333,0.2
7,0.832772,0.408025,0.512397,0.143440,0.009806,0.249847,0.0,0.421053,0.122807,0.000000,0.555556,0.081633,0.555556,0.458333,0.066667,0.0000,0.000000,0.0,0.500000,0.2
8,0.759035,0.558915,0.332940,0.189667,0.493694,0.795749,0.0,0.473684,0.122807,0.118644,0.444444,0.755102,0.555556,0.375000,0.333333,0.0000,0.000000,0.0,0.500000,0.4
9,0.216841,0.291890,0.645809,0.282121,0.411444,0.538955,0.0,0.210526,0.122807,0.118644,0.444444,0.112245,0.444444,0.305556,0.266667,0.0000,0.000000,0.0,0.500000,0.6


In [4]:
X = df_o.drop("isFirstDown",1)   #Feature Matrix
y = df_o["isFirstDown"]          #Target Variable

In [5]:
# from sklearn.model_selection import train_test_split # Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=72)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(31544, 19)
(13520, 19)
(31544,)
(13520,)


In [6]:
# Create Decision Tree classifer object
rfc = RandomForestClassifier(n_estimators=10,criterion="entropy", max_depth=4)

In [7]:
# Train Decision Tree Classifer
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [8]:
#Predict the response for test dataset
y_pred = rfc.predict(X_test)

In [9]:
# Model Accuracy, how often is the classifier correct?

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_pred)
print('F1 score: %f' % f1)

acc = pd.read_csv("../data/external/accuracies.csv", index_col=0)
acc.at['Random Forest, Oversampled', 'Accuracy'] = accuracy
acc.at['Random Forest, Oversampled', 'Precision'] = precision
acc.at['Random Forest, Oversampled', 'Recall'] = recall
acc.at['Random Forest, Oversampled', 'F1'] = f1
acc.to_csv("../data/external/accuracies.csv")

Accuracy: 0.722189
Precision: 0.813483
Recall: 0.583443
F1 score: 0.679522


In [10]:
#Present Confusion Matrix to show accuracy
print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.86      0.75      6695
           1       0.81      0.58      0.68      6825

   micro avg       0.72      0.72      0.72     13520
   macro avg       0.74      0.72      0.72     13520
weighted avg       0.74      0.72      0.72     13520
 
 [[5782  913]
 [2843 3982]]


In [11]:
features = list(X_train)
#features

In [12]:
dot_data = StringIO()  
export_graphviz(rfc.estimators_[0], out_file=dot_data,feature_names=features,filled=True,rounded=True,rotate=True)

import pydot
graph = pydot.graph_from_dot_data(dot_data.getvalue())  
Image(graph[0].create_png(), width=7860, height=2160)

ModuleNotFoundError: No module named 'pydot'